In [22]:
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime, time, date

# ========== MANUAL CHANGES ==========
Audit_File = ""
Sales_File = ""
Week_End_Date = ""

# ========== ALWAYS THE SAME ==========
# Normalize date/time format to ensure excel formulas don't error
def normalize_time(val):
    """Convert to datetime.time if possible, else return None."""
    if pd.isna(val):
        return None
    if isinstance(val, datetime):
        return val.time()
    if isinstance(val, time):
        return val
    try:
        return pd.to_datetime(val).time()
    except Exception:
        return None

def normalize_date(val):
    """Convert to datetime.date if possible, else return None."""
    if pd.isna(val):
        return None
    if isinstance(val, datetime):
        return val.date()
    if isinstance(val, date):
        return val
    try:
        return pd.to_datetime(val).date()
    except Exception:
        return None

# Sales file tabs to dfs
sales_base = pd.read_excel(Sales_File, sheet_name="Base Points", engine="openpyxl")
sales_bonus = pd.read_excel(Sales_File, sheet_name="Bonus Points", engine="openpyxl")

# Load audit workbook
wb = load_workbook(Audit_File)

# Collect results for "summary" tab
summary_data = []  

for sheet in wb.sheetnames:
    if sheet in ["Base Points", "Bonus Points", "Summary"]:
        continue
    
    ws = wb[sheet]

    # Base points
    lookup_val = ws["A1"].value
    if lookup_val:
        matches = sales_base[sales_base.iloc[:, 1] == lookup_val]  # Show name (column B)
        for r_idx, row in enumerate(matches.itertuples(index=False), start=14):
            for c_idx, val in enumerate(row[:21], start=1):  # A:U
                if c_idx == 4:  # Time (column D)
                    tval = normalize_time(val)
                    ws.cell(row=r_idx, column=c_idx).value = tval
                    ws.cell(row=r_idx, column=c_idx).number_format = "h:mm AM/PM"
                elif c_idx in [3, 5]:  # Date columns (C & E)
                    dval = normalize_date(val)
                    ws.cell(row=r_idx, column=c_idx).value = dval
                    ws.cell(row=r_idx, column=c_idx).number_format = "m/d/yy"
                else:
                    ws.cell(row=r_idx, column=c_idx).value = val

    # Bonus Points
    lookup_val_1 = ws["A2"].value
    lookup_val_2 = ws["A3"].value
    lookup_val_3 = ws["A4"].value
    if lookup_val_1 or lookup_val_2:
        matches = sales_bonus[sales_bonus.iloc[:, 14].isin([lookup_val_1, lookup_val_2, lookup_val_3])]  # Bonus name(s) (column O)
        for r_idx, row in enumerate(matches.itertuples(index=False), start=14):
            for c_idx, val in enumerate(row[:26], start=41):  # Bonus Data (AO:BN)
                if c_idx == 4:  # Time column
                    tval = normalize_time(val)
                    ws.cell(row=r_idx, column=c_idx).value = tval
                    ws.cell(row=r_idx, column=c_idx).number_format = "h:mm AM/PM"
                elif c_idx in [3, 5]:  # Date columns (AQ & AS)
                    dval = normalize_date(val)
                    ws.cell(row=r_idx, column=c_idx).value = dval
                    ws.cell(row=r_idx, column=c_idx).number_format = "m/d/yy"
                else:
                    ws.cell(row=r_idx, column=c_idx).value = val

    # Summary Formula
    ws["AL10"] = '=COUNTIFS(AL14:AL4000,TRUE,AI14:AI4000,"<>No",O14:O4000,"<>0")'

    # Collect result for summary
    summary_data.append((sheet, f"='{sheet}'!AL10"))

# Add Summary Tab
if "Summary" in wb.sheetnames:
    ws_summary = wb["Summary"]
    wb.remove(ws_summary)
ws_summary = wb.create_sheet("Summary")

ws_summary["A1"] = "Tab Name"
ws_summary["B1"] = "Count (AL10)"

for i, (tab, formula) in enumerate(summary_data, start=2):
    ws_summary[f"A{i}"] = tab
    ws_summary[f"B{i}"] = formula

# Add total
ws_summary[f"A{len(summary_data)+2}"] = "Total"
ws_summary[f"B{len(summary_data)+2}"] = f"=SUM(B2:B{len(summary_data)+1})"

# Save
wb.save(f"Bonus_Audit_{Week_End_Date}.xlsx")
print("Processing complete")

Processing complete -> Bonus_Audit_01.04.25.xlsx
